In [3]:
import setup

setup.init_django()

In [4]:
from decouple import config, AutoConfig
config = AutoConfig(search_path="/home/harry/chatbotDjango") 

In [5]:
from rag import db as rag_db, settings as rag_settings, engines as rag_engines

In [6]:
rag_settings.init()
semantic_query_engine = rag_engines.get_semantic_query_engine()
sql_query_engine = rag_engines.get_sql_query_engine(sql_only=True)

In [7]:
print_default_prompts = True

In [8]:
prompts_dict = sql_query_engine.get_prompts()

if print_default_prompts:
    print(prompts_dict['response_synthesis_prompt'].template)

Given an input question, synthesize a response from the query results.
Query: {query_str}
SQL: {sql_query}
SQL Response: {context_str}
Response: 



In [9]:
if print_default_prompts:
    print(sql_query_engine.sql_retriever.get_prompts()['text_to_sql_prompt'].template)

Given an input question, create a precise {dialect} PostgreSQL query to answer it. Follow these guidelines:

1. Use only tables and columns from the provided schema.
2. Select only relevant columns, never all columns.
3. Qualify column names with table names when necessary.
4. Use appropriate JOINs, WHERE clauses, and aggregations.
5. Order results to highlight the most pertinent information.
6. Avoid querying non-existent columns or tables.
7. Optimize the query for performance where possible.

Your response should contain only the SQL query, without any additional explanation or formatting. Do not use markdown or prepend the query with the term `sql`.

Schema:
{schema}

Question: {query_str}
SQL Query:



In [10]:
from llama_index.core import PromptTemplate

In [11]:
custom_sql_prompt = PromptTemplate("""Given an input question, create a precise {dialect} PostgreSQL query to answer it. Follow these guidelines:

1. Use only tables and columns from the provided schema.
2. Select only relevant columns, never all columns.
3. Qualify column names with table names when necessary.
4. Use appropriate JOINs, WHERE clauses, and aggregations.
5. Order results to highlight the most pertinent information.
6. Avoid querying non-existent columns or tables.
7. Optimize the query for performance where possible.

Your response should contain only the SQL query, without any additional explanation or formatting. Do not use markdown or prepend the query with the term `sql`.

Schema:
{schema}

Question: {query_str}
SQL Query:
""")

In [12]:
custom_response_synthesis_prompt = PromptTemplate("""Given an input question, synthesize a response from the query results.
Query: {query_str}
SQL: {sql_query}
SQL Response: {context_str}
Response: 
""")

In [13]:
custom_response_synthesis_prompt.template_vars

['query_str', 'sql_query', 'context_str']

In [14]:
sql_query_engine = rag_engines.get_sql_query_engine(
    sql_only=False,
    synthesize_response=True
)

In [15]:
print(sql_query_engine.sql_retriever.get_prompts()['text_to_sql_prompt'].template)

Given an input question, create a precise {dialect} PostgreSQL query to answer it. Follow these guidelines:

1. Use only tables and columns from the provided schema.
2. Select only relevant columns, never all columns.
3. Qualify column names with table names when necessary.
4. Use appropriate JOINs, WHERE clauses, and aggregations.
5. Order results to highlight the most pertinent information.
6. Avoid querying non-existent columns or tables.
7. Optimize the query for performance where possible.

Your response should contain only the SQL query, without any additional explanation or formatting. Do not use markdown or prepend the query with the term `sql`.

Schema:
{schema}

Question: {query_str}
SQL Query:



In [18]:
response = sql_query_engine.query("What blog post has the least views from today?")
print(str(response))

The blog post with the least views today is "Blog Post 4" with 540 views.


In [19]:
for node in response.source_nodes:
    print(node.node.get_content())

[(28, 'Blog Post 4', 540)]
